In [62]:
import tensorflow as tf
import numpy as np
import os
import time

In [63]:
from keras.layers import Dense,LSTM
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam

Read the data. Here I'm using the .txt file of my favorite book 'The Alchemist'.

First, look in the text:

In [64]:
# Read, then decode for py2 compat.
text = open('alchemist.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 324423 characters


In [65]:
# Take a look at the first 250 characters in text
print(text[:250])

The Project Gutenberg EBook of The Alchemist, by Ben Jonson

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg 


In [66]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [67]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [68]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

T
h
e
 
P


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [69]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'The Project Gutenberg EBook of The Alchemist, by Ben Jonson\r\n\r\nThis eBook is for the use of anyone an'
'ywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-'
'use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gu'
'tenberg.org\r\n\r\n\r\nTitle: The Alchemist\r\n\r\nAuthor: Ben Jonson\r\n\r\nRelease Date: May, 2003 [Etext #4081]\r'
'\nPosting Date: January 7, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK TH'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [70]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [71]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'The Project Gutenberg EBook of The Alchemist, by Ben Jonson\r\n\r\nThis eBook is for the use of anyone a'
Target data: 'he Project Gutenberg EBook of The Alchemist, by Ben Jonson\r\n\r\nThis eBook is for the use of anyone an'


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [72]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [73]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [74]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]),
                            tf.keras.layers.GRU(rnn_units, return_sequences=True,  stateful=True, recurrent_initializer='glorot_uniform'),
                            tf.keras.layers.Dense(vocab_size)
])
  return model

In [75]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [76]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 86) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [77]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (64, None, 256)           22016     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_7 (Dense)              (64, None, 86)            88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [78]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([51, 60, 22, 62, 45, 76, 62, 25, 31, 26,  0, 61, 81, 30, 26, 11,  3,
        1, 69, 59,  1, 44, 25, 52, 56, 27, 31, 18, 14, 27,  2, 24, 49, 65,
       24, 56, 56, 37, 31, 14, 61, 69, 57, 40, 27, 26, 67, 21, 57, 10,  4,
       49, 12, 24, 75, 39, 62, 54, 74,  4, 85,  8, 62,  1, 17, 20,  9, 40,
       14,  5, 61, 39, 10, 28, 48, 61, 36, 33, 71, 13, 50, 84, 26, 15,  0,
       34,  4, 62, 47, 64, 43, 69, 28, 22, 58, 25, 38, 78, 14, 43])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [79]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 86)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4552207


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [80]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [81]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In [82]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
50/50 [==============================] - 9s 130ms/step - loss: 3.5547 - accuracy: 0.1709
Epoch 2/10
50/50 [==============================] - 7s 128ms/step - loss: 2.5254 - accuracy: 0.3167
Epoch 3/10
50/50 [==============================] - 7s 127ms/step - loss: 2.2852 - accuracy: 0.3662
Epoch 4/10
50/50 [==============================] - 7s 126ms/step - loss: 2.1582 - accuracy: 0.3876
Epoch 5/10
50/50 [==============================] - 7s 127ms/step - loss: 2.0507 - accuracy: 0.4131
Epoch 6/10
50/50 [==============================] - 7s 128ms/step - loss: 1.9587 - accuracy: 0.4343
Epoch 7/10
50/50 [==============================] - 7s 128ms/step - loss: 1.8829 - accuracy: 0.4526
Epoch 8/10
50/50 [==============================] - 7s 127ms/step - loss: 1.8126 - accuracy: 0.4721
Epoch 9/10
50/50 [==============================] - 7s 131ms/step - loss: 1.7477 - accuracy: 0.4897
Epoch 10/10
50/50 [==============================] - 7s 128ms/step - loss: 1.6862 - accuracy: 0.5070

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [83]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_8 (Dense)              (1, None, 86)             88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [84]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 3000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [85]:
print(generate_text(model, start_string=u"egypt "))

egypt and evenblempinare them im thes tas hon,
  You, by coust, se.

  FACE. Nay, and  wee go Engive?

   NAVE. You say, "Thrizan That Sturk, sube,
  Whe pleas.
  That shis now!
   (Geest the feargel
(he  lang anding by bright thenedly Ase fene: them accoptake henes,
  This was meatine. He his mest
  Chere, a doct on:
  Turm is berought, arl'do?

  [AKATE.]

  [EXIT DRUG.]

  SUR. I huad inowar!

  DRUG. Ron hil cheraining.
  An prayk con?
  Whas you give these fear greer goar ip an the deaces;
  And'd me come that a candow, dewassix, nabue by cligtin,
  And wast do), and Mad.

  SUB. I sape hin not!
  It your 2pponce with pladam
  Shey have blears, so think you court
  Oix of BALNOD DERCONIT MOOST.]

  SUB. Adther.

  SUR. No, "hight,
  And lated it is somperan.

  A FACE. Thy inst ha, poetly: the gill:
  I that us shojelatice of passies the ushally passe be dough
  Which that?

  ANA. Are you pees this bees, and hast, bath you prekand,
  What you day the glaice; as and when.

  FACE.

###**Tuning the hyperparameters:**
##### Epochs *from* 10 to 25,
 ##### Batch Size *from* 64 to 128,
 ##### Learning Rate *from* 0.001 to 0.01, 
 ##### Adding LSTM layer instead of GRU.



In [86]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         activation='tanh', recurrent_activation='sigmoid',
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [87]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=128) #increased batchsize

In [88]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (128, None, 256)          22016     
_________________________________________________________________
lstm_6 (LSTM)                (128, None, 1024)         5246976   
_________________________________________________________________
dropout_6 (Dropout)          (128, None, 1024)         0         
_________________________________________________________________
dense_9 (Dense)              (128, None, 86)           88150     
Total params: 5,357,142
Trainable params: 5,357,142
Non-trainable params: 0
_________________________________________________________________


In [92]:
dataset= sequences.map(split_input_target)
dataset= dataset.shuffle(10000).batch(128, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

In [94]:
adam = Adam (learning_rate=0.01)
model.compile(optimizer=adam, loss=loss, metrics=['accuracy'])

In [95]:
EPOCHS= 25
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25
25/25 [==============================] - 9s 259ms/step - loss: 3.9494 - accuracy: 0.1481
Epoch 2/25
25/25 [==============================] - 7s 259ms/step - loss: 2.5285 - accuracy: 0.3133
Epoch 3/25
25/25 [==============================] - 7s 254ms/step - loss: 2.2185 - accuracy: 0.3758
Epoch 4/25
25/25 [==============================] - 7s 255ms/step - loss: 2.0611 - accuracy: 0.4106
Epoch 5/25
25/25 [==============================] - 7s 255ms/step - loss: 1.9469 - accuracy: 0.4386
Epoch 6/25
25/25 [==============================] - 7s 260ms/step - loss: 1.8546 - accuracy: 0.4602
Epoch 7/25
25/25 [==============================] - 7s 256ms/step - loss: 1.7822 - accuracy: 0.4782
Epoch 8/25
25/25 [==============================] - 7s 257ms/step - loss: 1.7238 - accuracy: 0.4933
Epoch 9/25
25/25 [==============================] - 7s 256ms/step - loss: 1.6708 - accuracy: 0.5077
Epoch 10/25
25/25 [==============================] - 7s 264ms/step - loss: 1.6291 - accuracy: 0.5186

In [96]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (1, None, 256)            22016     
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dropout_7 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
dense_10 (Dense)             (1, None, 86)             88150     
Total params: 5,357,142
Trainable params: 5,357,142
Non-trainable params: 0
_________________________________________________________________


In [98]:
print(generate_text(model, start_string=u"egypt "))

egypt of all, in Hebre of the employed to enfere.

TIRD, assonation.

PATURM, greated stope of a parts.

BRADKMOUS, smitte.

SHIGLLINE, pupply.

EANIANS, untimate.

FUCIPING, attendune, continuenated.

JAMPULDER, conception.

GROPE, whief.

EDGE, worn and conge.

MORRIST, reverse with the widown, a femf
what call like a round of promusting in master
  An mistress for your brother o'er he is very up. all my blazoner,
  To say my count help, and these thousand back. I'll do Drydle
  DOL. Not the will: you are to make you to keep
  No brave a sight, my heart of the best is that
courcual colours to go acsoritation or right.

  DAP. What, good sure it of a ring. Plays it?

  RUB. You found, then sui, Pissill'd, Ananias of the sister
  That discitable Nehend the worod of read person play
from their
will use of leplet alont of the saining or transanted creagrie.

DIGHTLE, relation.

BRAGANT, repair of caprice.

FACUL, becoverns more from sublimatice-vocour.

DISTART, wonder.

GANGANROLLY, I d

Here, the model did a decent job in generating valid text with accuracy of 62%. Hence, I'll add one more layer of LSTM with a couple of activation functions to tune it furthermore. Here I used tanh and sigmoid functions as activation and recurrent_activation functions.

### **Tuning furthermore..**
##### Epochs *from* 25 to 50,
 ##### Batch Size *from* 128 to 64,
 ##### Learning Rate *to its default*, 
 ##### Adding one more layer of LSTM




In [106]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         activation='tanh', recurrent_activation='sigmoid',
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(rnn_units,
                         activation='tanh', recurrent_activation='sigmoid',
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [107]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=64) #decreased batchsize

In [101]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (64, None, 256)           22016     
_________________________________________________________________
lstm_8 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dropout_8 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
lstm_9 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dropout_9 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense_11 (Dense)             (64, None, 86)            88150     
Total params: 13,749,846
Trainable params: 13,749,846
Non-trainable params: 0
_________________________________________

In [108]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [109]:
dataset= sequences.map(split_input_target)
dataset= dataset.shuffle(10000).batch(64, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [105]:
EPOCHS= 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
50/50 [==============================] - 22s 354ms/step - loss: 3.5259 - accuracy: 0.1500
Epoch 2/50
50/50 [==============================] - 18s 356ms/step - loss: 2.8848 - accuracy: 0.2348
Epoch 3/50
50/50 [==============================] - 19s 358ms/step - loss: 2.4658 - accuracy: 0.3207
Epoch 4/50
50/50 [==============================] - 19s 359ms/step - loss: 2.2301 - accuracy: 0.3677
Epoch 5/50
50/50 [==============================] - 19s 359ms/step - loss: 2.0903 - accuracy: 0.4001
Epoch 6/50
50/50 [==============================] - 19s 359ms/step - loss: 1.9807 - accuracy: 0.4278
Epoch 7/50
50/50 [==============================] - 19s 360ms/step - loss: 1.8830 - accuracy: 0.4519
Epoch 8/50
50/50 [==============================] - 19s 360ms/step - loss: 1.8014 - accuracy: 0.4724
Epoch 9/50
50/50 [==============================] - 19s 361ms/step - loss: 1.7265 - accuracy: 0.4937
Epoch 10/50
50/50 [==============================] - 19s 360ms/step - loss: 1.6621 - accura

In [110]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (1, None, 256)            22016     
_________________________________________________________________
lstm_12 (LSTM)               (1, None, 1024)           5246976   
_________________________________________________________________
dropout_12 (Dropout)         (1, None, 1024)           0         
_________________________________________________________________
lstm_13 (LSTM)               (1, None, 1024)           8392704   
_________________________________________________________________
dropout_13 (Dropout)         (1, None, 1024)           0         
_________________________________________________________________
dense_13 (Dense)             (1, None, 86)             88150     
Total params: 13,749,846
Trainable params: 13,749,846
Non-trainable params: 0
_________________________________________

In [111]:
print(generate_text(model, start_string=u"egypt "))

egypt that we do not necessarily
knowledger, Jonson's fellow, cheatmelousing and player says set
art in the plot us they sure.

  KAS. I will not.

  ANA. What is't you, sir?

  ANA. Locusts
  Of the fear hath placed him, and then all here have incidenels,
  They say, a wear doublets, term another, playwright
the letter do not solicit donations in locations
where we have not received written confirmation of compliance.  Jonson's birthpipance
ox the state of carrs of letters for puppets, with a
company made up of the choicest spirits of himself, and if at no court not in strizen, and for a widow,
  To furnish household.

  SUB. Excellent, more live; and for mind
  The sole of huro, and grieve me. The house, sir, does shop landuades spend that the
baws duch the satire of "Every Man Out of His Humour," and "Cynthia's Revels,"
Daniel under the characters Facenberg, sir.

  MAM. Will not you quit her in the house.

  SUB. How, Dol!

  FACE. She lies,
  This is my faith, to call him officers

This time, the model did an excellent job in generating the exact text with an accuracy of 93%.